In [1]:
# use SQLite to create a database

import DBcm

db_details = "Swimclub.sqlite3"

In [ ]:
# load a file

import os

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

In [ ]:
first = files[0]
name, age, _, _ = first.removesuffix(".txt").split("-")

In [ ]:
# create 3 tables to store the information

## check the current tables
with DBcm.UseDatabase(db_details) as db:
    db.execute("pragma table_list")
    results = db.fetchall()
results


## create tables
with DBcm.UseDatabase(db_details) as db:
    SQL_1 = """
        create table if not exists swimmers (
            id integer not null primary key autoincrement,
            name varchar(32) not null,
            age integer not null
        )
    """

    SQL_2 = """
        create table if not exists times (
            swimmer_id integer not null,
            event_id integer not null,
            time varchar(16) not null,
            ts timestamp default current_timestamp
        )
    """

    SQL_3 = """
        create table if not exist events(
            id integer not null primary key autoincrement,
            distance varchar(16) not null,
            stroke varchar(16) not null
        )
    """
    db.execute(SQL_1)
    db.execute(SQL_2)
    db.execute(SQL_3)

In [ ]:
# SQL query: insert data
SQL_INSERT = """
    insert into swimmers
    (name,age)
    values
    (?,?)
"""

# execute the command using DBcm.UseDatabase
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_INSERT,(name,age,))

In [ ]:
# SQL query: select data

SQL_SELECT = """select * from swimmers"""
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_SELECT)
    results = db.fetchall()

results

In [ ]:
# SQL query: delete data

SQL_DELETE = """delete from swimmers"""
with DBcm.UseDatabase(db_details) as db:
    db.execute(SQL_DELETE)



In [ ]:
# Count the number of items
SQL_COUNT = """select count(*) from times"""

# Select the top 10 items
SQL_TOPTEN = """select * from times limit 10"""


In [ ]:
# Batch insert data from a folder and avoid repetition
# Populate the "swimmers" table.
import os
import DBcm

db_details = "Swimclub.sqlite3"

FOLDER = "swimdata/"

files = os.listdir(FOLDER)
files.remove(".DS_Store")

# selection based on the condition
SQL_SELECT = """
    select * from swimmers
    where name = ? and age = ?
"""

SQL_INSERT = """
    insert into swimmers
    (name, age)
    values
    (?,?)
"""


with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, _, _ = fn.removesuffix(".txt").split("-")
        db.execute(SQL_SELECT, (name, age,)) # NOTICE: tuple is required; therefore, the second comma is needed. 
        if db.fetchall():
            continue # break the current loop and continue with the next item
        db.execute(SQL_INSERT, (name, age,))


# Populate the "events" table following the same steps.

In [ ]:
# Populate the "times" table with data from "swimmers" and "events" 
SQL_GET_SWIMMER = """
    select id from swimmers
    where name = ? and age = ?
"""

SQL_GET_EVENT = """
    select id from events
    where distance = ? and stroke = ?
"""

SQL_INSERT = """
    insert into times
    (swimmer_id, event_id, time)
    values
    (?, ?, ?)
"""

with DBcm.UseDatabase(db_details) as db:
    for fn in files:
        name, age, distance, stroke = fn.removesuffix(".txt").split("-")
        db.execute(SQL_GET_SWIMMER, (name,age,))
        s_id = db.fetchone()[0] # otbained the id of the swimmer
        db.execute(SQL_GET_EVENT,(distance,stroke,))
        e_id = db.fetchon()[0]  # otbained the id of the event
        with open(FOLDER+fn) as sf:
            times = sf.read().strip().split(",")
            for t in times:
                db.execute(SQL_INSERT, (s_id, e_id, t,))